This scorecard tracks target vs actuals for QTD.

It contains:
1. Construction funded data as plan build (XGS/MG)
2. LU Enablement data (XGS/MG)
3. Sales data (XGS/MG)

## Data import notes:
##### 1. Construction funding info to the end of the year from PowerBI:
###### - For monthly targets - pull once at beginning of the month
###### - For QTD report - pull once a quarter, two weeks before the start of the quarter
##### 2.  Report directly from Salesforce

In [88]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

ProgrammingError: Oracle Client library has already been initialized

In [2]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

## Getting enablement data

In [3]:
# XGS qualifications count at wire center level - manually update date filters every month and at the beginning of the year

query = """
            WITH QD AS (
            SELECT  
                    O.LUID --count distinct LU id's
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    ,case when O.UNI_ECD >= '1-OCT-2022' and O.UNI_ECD < '1-NOV-2022' then 'Y' end as MONTH_CNT_YN
                    ,case when O.UNI_ECD >= '1-JAN-2022' and O.UNI_ECD < '1-JAN-2023' then 'Y' end as YEAR_CNT_YN
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,count(distinct case when MONTH_CNT_YN = 'Y' then LUID end) as monthly_xgs_enabled_cnt
                    ,count(distinct case when YEAR_CNT_YN = 'Y' then LUID end) as ytd_xgs_enabled_cnt
                    ,count(distinct LUID) as cumulative_xgs_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,MONTH_CNT_YN
                    ,YEAR_CNT_YN
            

                                """

query2 = """
            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_POLYGONID
                    ,O.UNI_SERVICETYPE
                    ,O.UNI_STATUS
                    ,O.TECH_TYPE
                    ,O.FSAI
                    ,O.OLT_TYPE
                    ,O.ADDRESS
                    ,O.OLT
                    ,O.UNI_ECD
                    --O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND uni_ecd >= '1-JAN-2022' --'1-JAN-2023' 
            )
            
            SELECT  *
                    
            FROM    QD

                                """

df_ora = pd.read_sql(query, con=conn)
df_ora_details = pd.read_sql(query2, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [191]:
%%time

# count of MG enabled LUs at wire center level - manually update date filters
# 10/10/22 - update multigig enablement date to reflect mg turn up date

mg_query = """
           
           WITH MG_DATES AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    ,case when O.UNI_POLYGONID in ('1179498','1179547','1179121','1179499') then to_date('30-SEP-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179687','1179674') then to_date('10-OCT-2022', 'DD-MON-YYYY')
                          else O.UNI_ECD end
                          AS UNI_ECD
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in ('1178832','1178694','1178144',
                                          '1179498','1179547','1179121','1179499',  --Phoenix AZ turned up 9/30/22
                                          '1179687','1179674' -- Golden CO turned up 10/10/22
                                          ) --add in MG polygon IDs here
                    
            ),
            QD AS (
            
            SELECT  LUID
                    ,STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,ADDRESS
                    ,UNI_SERVICETYPE
                    ,OLT
                    ,TECH_TYPE
                    ,OLT_TYPE
                    ,FSAI
                    ,UNI_POLYGONID
                    ,UNI_STATUS
                    ,UNI_NDSJOBNUMBER
                    ,UNI_FWRKSJOBNUMBER
                    ,case when UNI_ECD >= '1-OCT-2022' and UNI_ECD < '1-NOV-2022' then 'Y' end as MONTH_CNT_YN
                    ,case when UNI_ECD >= '1-JAN-2022' and UNI_ECD < '1-JAN-2023' then 'Y' end as YEAR_CNT_YN
                    
            FROM MG_DATES
            
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,count(distinct case when MONTH_CNT_YN = 'Y' then LUID end) as monthly_mg_enabled_cnt
                    ,count(distinct case when YEAR_CNT_YN = 'Y' then LUID end) as ytd_mg_enabled_cnt
                    ,count(distinct LUID) as cumulative_mg_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,MONTH_CNT_YN
                    ,YEAR_CNT_YN


"""

mg_lu_ora = pd.read_sql(mg_query, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 0 ns
Wall time: 742 ms


In [192]:
mg_lu_ora

,STATE,PRIMARY_WIRE_CENTER_ID,MONTHLY_MG_ENABLED_CNT,YTD_MG_ENABLED_CNT,CUMULATIVE_MG_ENABLED_CNT
0,AZ,PHNXAZNE,0,212,212
1,AZ,PHNXAZSO,0,850,850
2,CO,ARVDCOMA,0,196,196
3,CO,GLDNCOMA,341,341,341
4,MN,MPLSMNBE,0,336,336
5,WA,PYLPWA01,0,342,342


In [193]:
# sum queries at wire center level - add mg flag according to mg enablement or core network status
xgs_enablement_wc = df_ora.groupby(['STATE','PRIMARY_WIRE_CENTER_ID'], as_index = False).sum()
mg_enablement_wc = mg_lu_ora.groupby(['STATE','PRIMARY_WIRE_CENTER_ID'], as_index = False).sum()

In [194]:
# merge xgs and mg enablement
luqual_enablement_df = xgs_enablement_wc.merge(mg_enablement_wc, how = 'outer', on = ['STATE','PRIMARY_WIRE_CENTER_ID']).fillna(0)
luqual_enablement_df_save = luqual_enablement_df.copy() #save copy

# check data
luqual_enablement_df[luqual_enablement_df['CUMULATIVE_MG_ENABLED_CNT']>0]

,STATE,PRIMARY_WIRE_CENTER_ID,MONTHLY_XGS_ENABLED_CNT,YTD_XGS_ENABLED_CNT,CUMULATIVE_XGS_ENABLED_CNT,MONTHLY_MG_ENABLED_CNT,YTD_MG_ENABLED_CNT,CUMULATIVE_MG_ENABLED_CNT
0,AZ,PHNXAZNE,0,212,212,0.0,212.0,212.0
1,AZ,PHNXAZSO,0,5544,5793,0.0,850.0,850.0
8,CO,ARVDCOMA,0,196,196,0.0,196.0,196.0
14,CO,GLDNCOMA,0,341,341,341.0,341.0,341.0
22,MN,MPLSMNBE,0,2417,2417,0.0,336.0,336.0
30,WA,PYLPWA01,0,342,342,0.0,342.0,342.0


In [195]:
# add core status network
multigig_ready_wirecenters_df = pd.read_excel(r'Inputs/081622_OpportunityScore_MultiGig_Readiness.xlsx',sheet_name = 'Datasheet')
multigig_ready_wirecenters_df_save = multigig_ready_wirecenters_df.copy() # save for export in details
multigig_ready_wirecenters_df = multigig_ready_wirecenters_df[['STATE','PRIMARY_WIRE_CENTER_ID','Core Status']]
multigig_ready_wirecenters_df = multigig_ready_wirecenters_df.merge(luqual_enablement_df, how = 'outer', on = ['STATE','PRIMARY_WIRE_CENTER_ID'])
# set core status of wire centers with actual MG enablement to MultiGIG, regardless of worksheet status
multigig_ready_wirecenters_df.loc[multigig_ready_wirecenters_df['CUMULATIVE_MG_ENABLED_CNT']>0,'Core Status'] = 'MultiGIG'

In [196]:
# check data - use this as base to join to construction funded data
multigig_ready_wirecenters_df = multigig_ready_wirecenters_df.fillna(0)
multigig_ready_wirecenters_df[multigig_ready_wirecenters_df['CUMULATIVE_MG_ENABLED_CNT']>0]

,STATE,PRIMARY_WIRE_CENTER_ID,Core Status,MONTHLY_XGS_ENABLED_CNT,YTD_XGS_ENABLED_CNT,CUMULATIVE_XGS_ENABLED_CNT,MONTHLY_MG_ENABLED_CNT,YTD_MG_ENABLED_CNT,CUMULATIVE_MG_ENABLED_CNT
47,AZ,PHNXAZNE,MultiGIG,0.0,212.0,212.0,0.0,212.0,212.0
74,AZ,PHNXAZSO,MultiGIG,0.0,5544.0,5793.0,0.0,850.0,850.0
87,CO,ARVDCOMA,MultiGIG,0.0,196.0,196.0,0.0,196.0,196.0
125,WA,PYLPWA01,MultiGIG,0.0,342.0,342.0,0.0,342.0,342.0
126,MN,MPLSMNBE,MultiGIG,0.0,2417.0,2417.0,0.0,336.0,336.0
147,CO,GLDNCOMA,MultiGIG,0.0,341.0,341.0,341.0,341.0,341.0


## Getting QTD(YTD) and Monthly Construction Funded data

In [197]:
#Import Funding data from PowerBI
Construction_Funding_df = pd.read_excel(r'FundingDetailsExport/Quarterly/Details_Export_Table.xlsx',sheet_name = 'Export')
Construction_Funding_df_save = Construction_Funding_df.copy() # saving a copy for export later

In [198]:
# clean data
Construction_Funding_df = Construction_Funding_df[['STATE','PRIMARY_WIRE_CENTER_ID','PHASE_ECD_LOGIC','PHASE_NUM_OF_LUS']]
Construction_Funding_df['PHASE_ECD_LOGIC'] = Construction_Funding_df['PHASE_ECD_LOGIC'].dt.to_period('M')
Construction_Funding_df['PHASE_ECD_LOGIC'] = Construction_Funding_df['PHASE_ECD_LOGIC'].astype(str)
Construction_pivot = Construction_Funding_df.groupby(by = ["STATE","PRIMARY_WIRE_CENTER_ID","PHASE_ECD_LOGIC"], as_index = False).sum()
Construction_pivot = Construction_pivot.pivot(index=['STATE','PRIMARY_WIRE_CENTER_ID'], columns='PHASE_ECD_LOGIC')['PHASE_NUM_OF_LUS'].reset_index()

In [199]:
# check data
Construction_pivot = Construction_pivot.fillna(0)
Construction_pivot.head()

PHASE_ECD_LOGIC,STATE,PRIMARY_WIRE_CENTER_ID,2022-09,2022-10,2022-11,2022-12
0,AL,LSVLALXA,0.0,0.0,0.0,142.0
1,AR,DWTTARXA,0.0,0.0,0.0,70.0
2,AR,STTGARXB,0.0,0.0,0.0,40.0
3,AZ,TCSNAZFW,0.0,160.0,248.0,306.0
4,AZ,TCSNAZMA,0.0,1572.0,2517.0,1642.0


In [200]:
# add month and QTD (2023 update to YTD) columns - manually adjust date field names
Construction_pivot['Month_Funding'] = Construction_pivot['2022-10']
Construction_pivot['QTD_Funding'] = Construction_pivot['2022-10'] + Construction_pivot['2022-11'] + Construction_pivot['2022-12']
Construction_pivot['YTD_Funding'] = Construction_pivot['2022-10'] + Construction_pivot['2022-11'] + Construction_pivot['2022-12']

In [201]:
wirecenter_scorecard_details = Construction_pivot[['STATE','PRIMARY_WIRE_CENTER_ID','Month_Funding','QTD_Funding','YTD_Funding']].merge(multigig_ready_wirecenters_df, how = 'outer', on = ['STATE','PRIMARY_WIRE_CENTER_ID'])

In [202]:
# create month/qtd/ytd funding for mg only
wirecenter_scorecard_details['Month_Funding_mg_only'] = np.where(wirecenter_scorecard_details['Core Status']=='MultiGIG', wirecenter_scorecard_details['Month_Funding'],0)
wirecenter_scorecard_details['QTD_Funding_mg_only'] = np.where(wirecenter_scorecard_details['Core Status']=='MultiGIG', wirecenter_scorecard_details['QTD_Funding'],0)
wirecenter_scorecard_details['YTD_Funding_mg_only'] = np.where(wirecenter_scorecard_details['Core Status']=='MultiGIG', wirecenter_scorecard_details['YTD_Funding'],0) 

In [203]:
wirecenter_scorecard_details = wirecenter_scorecard_details[['STATE','PRIMARY_WIRE_CENTER_ID','Month_Funding','Month_Funding_mg_only','QTD_Funding','QTD_Funding_mg_only','YTD_Funding','YTD_Funding_mg_only','MONTHLY_XGS_ENABLED_CNT','YTD_XGS_ENABLED_CNT','CUMULATIVE_XGS_ENABLED_CNT','MONTHLY_MG_ENABLED_CNT','YTD_MG_ENABLED_CNT','CUMULATIVE_MG_ENABLED_CNT']].fillna(0)

In [204]:
# check data
wirecenter_scorecard_details[wirecenter_scorecard_details['CUMULATIVE_MG_ENABLED_CNT']>0]

,STATE,PRIMARY_WIRE_CENTER_ID,Month_Funding,Month_Funding_mg_only,QTD_Funding,QTD_Funding_mg_only,YTD_Funding,YTD_Funding_mg_only,MONTHLY_XGS_ENABLED_CNT,YTD_XGS_ENABLED_CNT,CUMULATIVE_XGS_ENABLED_CNT,MONTHLY_MG_ENABLED_CNT,YTD_MG_ENABLED_CNT,CUMULATIVE_MG_ENABLED_CNT
15,CO,GLDNCOMA,89.0,89.0,89.0,89.0,89.0,89.0,0.0,341.0,341.0,341.0,341.0,341.0
33,MN,MPLSMNBE,954.0,954.0,3718.0,3718.0,3718.0,3718.0,0.0,2417.0,2417.0,0.0,336.0,336.0
51,WA,PYLPWA01,0.0,0.0,400.0,400.0,400.0,400.0,0.0,342.0,342.0,0.0,342.0,342.0
92,AZ,PHNXAZNE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,212.0,212.0,0.0,212.0,212.0
113,AZ,PHNXAZSO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5544.0,5793.0,0.0,850.0,850.0
122,CO,ARVDCOMA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196.0,196.0,0.0,196.0,196.0


## Summarize wire center scorecard detail at state level to calculate sales rates

In [205]:
wirecenter_scorecard_state_summary = wirecenter_scorecard_details.groupby(['STATE'], as_index = False).sum()

In [206]:
# exclude states and order - manually adjust states
# manually filtering out AL, AR, VA
state_summary_mask = (wirecenter_scorecard_state_summary['STATE'] != 'AL')&(wirecenter_scorecard_state_summary['STATE'] != 'AR')&(wirecenter_scorecard_state_summary['STATE'] != 'VA') \
                    & (wirecenter_scorecard_state_summary['YTD_Funding'] > 0)

wirecenter_scorecard_state_summary = wirecenter_scorecard_state_summary[state_summary_mask]

In [207]:
# re-order states by mg enablement/construction funded
wirecenter_scorecard_state_summary = wirecenter_scorecard_state_summary.sort_values(['CUMULATIVE_MG_ENABLED_CNT','YTD_Funding'], ascending = False)

In [208]:
wirecenter_scorecard_state_summary

,STATE,Month_Funding,Month_Funding_mg_only,QTD_Funding,QTD_Funding_mg_only,YTD_Funding,YTD_Funding_mg_only,MONTHLY_XGS_ENABLED_CNT,YTD_XGS_ENABLED_CNT,CUMULATIVE_XGS_ENABLED_CNT,MONTHLY_MG_ENABLED_CNT,YTD_MG_ENABLED_CNT,CUMULATIVE_MG_ENABLED_CNT
2,AZ,2854.0,2694.0,8610.0,7896.0,8610.0,7896.0,582.0,11599.0,11848.0,0.0,1062.0,1062.0
3,CO,8724.0,423.0,18174.0,547.0,18174.0,547.0,0.0,5423.0,5573.0,341.0,537.0,537.0
13,WA,1478.0,633.0,6305.0,2170.0,6305.0,2170.0,0.0,1929.0,1929.0,0.0,342.0,342.0
7,MN,3384.0,1874.0,9718.0,6685.0,9718.0,6685.0,0.0,4483.0,4483.0,0.0,336.0,336.0
4,FL,13032.0,0.0,26952.0,0.0,26952.0,0.0,0.0,13579.0,13665.0,0.0,0.0,0.0
11,UT,1847.0,0.0,8656.0,2133.0,8656.0,2133.0,57.0,810.0,810.0,0.0,0.0,0.0
5,IA,0.0,0.0,248.0,0.0,248.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,NV,83.0,0.0,135.0,0.0,135.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Getting all-time sales data

In [209]:
# import salesforce report
salesforce_report_details = pd.read_excel(r'SalesforceReportExport/salesdata.xlsx',sheet_name = 'Ella XGS Sales') # this contains salesforce report

C:\Users\AD21236\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [210]:
# filter for only active subscribers
salesforce_summary = salesforce_report_details[salesforce_report_details['Account Status'] == 'Active']

In [211]:
# check speeds
salesforce_summary['Purchased Data Rate'].value_counts()

940M/940M    169
200M/200M     27
1G/1G         19
3G/3G          2
Name: Purchased Data Rate, dtype: int64

In [212]:
# add mg/non-mg column
salesforce_summary['MG'] = np.where(salesforce_summary['Purchased Data Rate'].str.contains('G'),'Y','N')

C:\Windows\Temp\ipykernel_18000\457290610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salesforce_summary['MG'] = np.where(salesforce_summary['Purchased Data Rate'].str.contains('G'),'Y','N')


In [213]:
# state summary of active xgs accounts

salesforce_state_summary_xgs = salesforce_summary[['Service State Short','MG','Order Number']].groupby('Service State Short', as_index = False)['Order Number'].nunique().rename(columns = {'Order Number': 'Active XGS Subscriptions'})

In [214]:
# state summary of active mg accounts

salesforce_state_summary_mg = salesforce_summary.loc[salesforce_summary['MG'] == 'Y',['Service State Short','MG','Order Number']].groupby('Service State Short', as_index = False)['Order Number'].nunique().rename(columns = {'Order Number': 'Active MG Subscriptions'})

In [215]:
salesforce_state_summary_xgs_mg = salesforce_state_summary_xgs.merge(salesforce_state_summary_mg, how = 'left', on = 'Service State Short').fillna(0)

In [216]:
# check data
salesforce_state_summary_xgs_mg = salesforce_state_summary_xgs_mg.rename(columns = {'Service State Short': 'STATE'})
salesforce_state_summary_xgs_mg

,STATE,Active XGS Subscriptions,Active MG Subscriptions
0,AZ,57,0.0
1,CO,25,9.0
2,FL,96,0.0
3,MN,17,3.0
4,WA,20,9.0


## Combine and calculate penetration rates

In [217]:
sales_penetration_state_summary = wirecenter_scorecard_state_summary.merge(salesforce_state_summary_xgs_mg, on = 'STATE', how = 'left')

In [218]:
sales_penetration_state_summary['XGSPON_Penetration_Rate'] = sales_penetration_state_summary['Active XGS Subscriptions'] / sales_penetration_state_summary['CUMULATIVE_XGS_ENABLED_CNT']
sales_penetration_state_summary['Multigig_Penetration_Rate'] = sales_penetration_state_summary['Active MG Subscriptions'] / sales_penetration_state_summary['CUMULATIVE_MG_ENABLED_CNT']

In [219]:
sales_penetration_state_summary = sales_penetration_state_summary.fillna(0)

In [220]:
sales_penetration_state_summary = sales_penetration_state_summary.sort_values(['Multigig_Penetration_Rate'], ascending = False)

## Create final view

In [221]:
multigig_venn_view = sales_penetration_state_summary.copy()

In [222]:
# create Grand Total row

multigig_venn_view.loc[len(multigig_venn_view.index)] = ['TOTAL',
                                                       multigig_venn_view['Month_Funding'].sum(),
                                                       multigig_venn_view['Month_Funding_mg_only'].sum(),
                                                       multigig_venn_view['QTD_Funding'].sum(),
                                                       multigig_venn_view['QTD_Funding_mg_only'].sum(),
                                                       multigig_venn_view['YTD_Funding'].sum(),
                                                       multigig_venn_view['YTD_Funding_mg_only'].sum(),
                                                       multigig_venn_view['MONTHLY_XGS_ENABLED_CNT'].sum(),
                                                       multigig_venn_view['YTD_XGS_ENABLED_CNT'].sum(),
                                                       multigig_venn_view['CUMULATIVE_XGS_ENABLED_CNT'].sum(),
                                                       multigig_venn_view['MONTHLY_MG_ENABLED_CNT'].sum(),
                                                       multigig_venn_view['YTD_MG_ENABLED_CNT'].sum(),
                                                       multigig_venn_view['CUMULATIVE_MG_ENABLED_CNT'].sum(),
                                                       multigig_venn_view['Active XGS Subscriptions'].sum(),
                                                       multigig_venn_view['Active MG Subscriptions'].sum(),
                                                       multigig_venn_view['Active XGS Subscriptions'].sum() / multigig_venn_view['CUMULATIVE_XGS_ENABLED_CNT'].sum(),
                                                       multigig_venn_view['Active MG Subscriptions'].sum() / multigig_venn_view['CUMULATIVE_MG_ENABLED_CNT'].sum()]


In [223]:
#QA data here
multigig_venn_view

,STATE,Month_Funding,Month_Funding_mg_only,QTD_Funding,QTD_Funding_mg_only,YTD_Funding,YTD_Funding_mg_only,MONTHLY_XGS_ENABLED_CNT,YTD_XGS_ENABLED_CNT,CUMULATIVE_XGS_ENABLED_CNT,MONTHLY_MG_ENABLED_CNT,YTD_MG_ENABLED_CNT,CUMULATIVE_MG_ENABLED_CNT,Active XGS Subscriptions,Active MG Subscriptions,XGSPON_Penetration_Rate,Multigig_Penetration_Rate
2,WA,1478.0,633.0,6305.0,2170.0,6305.0,2170.0,0.0,1929.0,1929.0,0.0,342.0,342.0,20.0,9.0,0.010368,0.026316
1,CO,8724.0,423.0,18174.0,547.0,18174.0,547.0,0.0,5423.0,5573.0,341.0,537.0,537.0,25.0,9.0,0.004486,0.016760
3,MN,3384.0,1874.0,9718.0,6685.0,9718.0,6685.0,0.0,4483.0,4483.0,0.0,336.0,336.0,17.0,3.0,0.003792,0.008929
0,AZ,2854.0,2694.0,8610.0,7896.0,8610.0,7896.0,582.0,11599.0,11848.0,0.0,1062.0,1062.0,57.0,0.0,0.004811,0.000000
4,FL,13032.0,0.0,26952.0,0.0,26952.0,0.0,0.0,13579.0,13665.0,0.0,0.0,0.0,96.0,0.0,0.007025,0.000000
5,UT,1847.0,0.0,8656.0,2133.0,8656.0,2133.0,57.0,810.0,810.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
6,IA,0.0,0.0,248.0,0.0,248.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
7,NV,83.0,0.0,135.0,0.0,135.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
8,TOTAL,31402.0,5624.0,78798.0,19431.0,78798.0,19431.0,639.0,37823.0,38308.0,341.0,2277.0,2277.0,215.0,21.0,0.005612,0.009223


In [224]:
# add in weekly goal columns

# Month_Funding / 4
# QTD_Funding / 12
multigig_venn_view['XGS_Enablement_Weekly_Goal'] = (multigig_venn_view['QTD_Funding'] / 12).astype(int)
multigig_venn_view['MG_Enablement_Weekly_Goal'] = (multigig_venn_view['QTD_Funding_mg_only'] / 12).astype(int)

In [231]:
# create calculated weekly average enablement - update denominator integer according to the number of full weeks passed

multigig_venn_view['XGS_Enablement_Avg_Weekly_Actual'] = (multigig_venn_view['MONTHLY_XGS_ENABLED_CNT'] / 1).astype(int)
multigig_venn_view['MG_Enablement_Avg_Weekly_Actual'] = (multigig_venn_view['MONTHLY_MG_ENABLED_CNT'] / 1).astype(int)

In [232]:
# reorder view for ppt
multigig_venn_view_final = multigig_venn_view[['STATE','QTD_Funding','QTD_Funding_mg_only','Month_Funding','Month_Funding_mg_only', \
                    'CUMULATIVE_XGS_ENABLED_CNT','CUMULATIVE_MG_ENABLED_CNT', \
                    'MONTHLY_XGS_ENABLED_CNT','MONTHLY_MG_ENABLED_CNT', \
                    'XGS_Enablement_Weekly_Goal','XGS_Enablement_Avg_Weekly_Actual', \
                    'MG_Enablement_Weekly_Goal','MG_Enablement_Avg_Weekly_Actual', \
                    'Active XGS Subscriptions','Active MG Subscriptions', \
                    'XGSPON_Penetration_Rate','Multigig_Penetration_Rate']]

In [233]:
# QA and export
multigig_venn_view_final

,STATE,QTD_Funding,QTD_Funding_mg_only,Month_Funding,Month_Funding_mg_only,CUMULATIVE_XGS_ENABLED_CNT,CUMULATIVE_MG_ENABLED_CNT,MONTHLY_XGS_ENABLED_CNT,MONTHLY_MG_ENABLED_CNT,XGS_Enablement_Weekly_Goal,XGS_Enablement_Avg_Weekly_Actual,MG_Enablement_Weekly_Goal,MG_Enablement_Avg_Weekly_Actual,Active XGS Subscriptions,Active MG Subscriptions,XGSPON_Penetration_Rate,Multigig_Penetration_Rate
2,WA,6305.0,2170.0,1478.0,633.0,1929.0,342.0,0.0,0.0,525,0,180,0,20.0,9.0,0.010368,0.026316
1,CO,18174.0,547.0,8724.0,423.0,5573.0,537.0,0.0,341.0,1514,0,45,341,25.0,9.0,0.004486,0.016760
3,MN,9718.0,6685.0,3384.0,1874.0,4483.0,336.0,0.0,0.0,809,0,557,0,17.0,3.0,0.003792,0.008929
0,AZ,8610.0,7896.0,2854.0,2694.0,11848.0,1062.0,582.0,0.0,717,582,658,0,57.0,0.0,0.004811,0.000000
4,FL,26952.0,0.0,13032.0,0.0,13665.0,0.0,0.0,0.0,2246,0,0,0,96.0,0.0,0.007025,0.000000
5,UT,8656.0,2133.0,1847.0,0.0,810.0,0.0,57.0,0.0,721,57,177,0,0.0,0.0,0.000000,0.000000
6,IA,248.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,0,0,0,0.0,0.0,0.000000,0.000000
7,NV,135.0,0.0,83.0,0.0,0.0,0.0,0.0,0.0,11,0,0,0,0.0,0.0,0.000000,0.000000
8,TOTAL,78798.0,19431.0,31402.0,5624.0,38308.0,2277.0,639.0,341.0,6566,639,1619,341,215.0,21.0,0.005612,0.009223


In [234]:
import time

TodaysDate = time.strftime("%m%d%y")

multigig_venn_view_final.to_excel("Output/" + TodaysDate +"_PathToPremierYTD.xlsx", sheet_name='YTD', index = False)